In [3]:
import time
import os
import cv2
import numpy as np
import pandas as pd
import glob
import shutil
from matplotlib import image
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm


from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from tqdm import tqdm

In [4]:
reloaded = tf.keras.models.load_model(
  './1671592035.h5', 
  # `custom_objects` tells keras how to load a `hub.KerasLayer`
  #custom_objects={'KerasLayer': hub.KerasLayer}
)

reloaded.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 37, 37, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 18, 18, 64)       0

In [6]:
base_dir = '../plant-seedlings-classification/data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
batch_size = 100
IMG_SHAPE = 150 

In [27]:
image_gen_val = ImageDataGenerator(rescale=1./255)

train_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=train_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='sparse')

Found 3801 images belonging to 12 classes.


In [28]:
predictions = reloaded.predict(train_data_gen)

39/39 [==============================] - 44s 1s/step


In [16]:
for dirname, _, filenames in os.walk(train_dir):
    for filename in filenames:
        os.path.join(dirname, filename)

In [19]:
DATADIR = train_dir
classes = os.listdir(train_dir)
IMG_SIZE=100
CATEGORIES = classes

In [13]:
for category in CATEGORIES:
    path=os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array=cv2.imread(os.path.join(path,img))
        break
    break

In [20]:
train_data=[]
def create_training_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                train_data.append([new_array,class_num])
            except Exception as e:
                pass
create_training_data()

In [22]:
lenofimage = len(train_data)
lenofimage

3801

In [32]:
X=[]
y=[]

for categories, label in train_data:
    X.append(categories)
    y.append(label)
##X = tf.keras.utils.normalize(X, axis = 1)

In [33]:
y=np.array(y)

In [30]:
predictions.shape

(3801, 12)

In [34]:
y.shape

(3801,)

In [35]:
svc = SVC(kernel='linear',gamma='auto')
svc.fit(predictions, y)

SVC(gamma='auto', kernel='linear')

In [37]:
for dirname, _, filenames in os.walk(val_dir):
    for filename in filenames:
        os.path.join(dirname, filename)

DATADIR = val_dir
classes = os.listdir(val_dir)
IMG_SIZE=100
CATEGORIES = classes

for category in CATEGORIES:
    path=os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array=cv2.imread(os.path.join(path,img))
        break
    break

val_data=[]
def create_val_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                val_data.append([new_array,class_num])
            except Exception as e:
                pass
create_val_data()

lenofimage = len(val_data)
lenofimage

X_test=[]
y_test=[]

for categories, label in val_data:
    X_test.append(categories)
    y_test.append(label)
##X = tf.keras.utils.normalize(X, axis = 1)

In [40]:
val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=val_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='sparse')

Found 949 images belonging to 12 classes.


In [41]:
y2 = reloaded.predict(val_data_gen)

10/10 [==============================] - 10s 1s/step


In [43]:
y2 = svc.predict(y2)

In [44]:
y2

array([6, 3, 6, 3, 6, 6, 6, 6, 6, 3, 6, 6, 6, 6, 6, 6, 6, 6, 3, 6, 6, 6,
       3, 6, 6, 6, 6, 6, 6, 3, 3, 6, 6, 6, 6, 3, 6, 3, 6, 6, 6, 6, 6, 6,
       3, 6, 6, 6, 3, 6, 6, 6, 6, 6, 6, 6, 3, 6, 6, 6, 6, 6, 3, 6, 6, 3,
       6, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 6, 6, 6, 3, 6,
       6, 6, 6, 3, 6, 3, 6, 6, 6, 3, 6, 6, 6, 3, 6, 3, 6, 3, 6, 6, 3, 6,
       3, 6, 3, 6, 6, 6, 3, 3, 6, 6, 6, 6, 3, 6, 6, 6, 6, 6, 6, 6, 3, 6,
       6, 6, 6, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 6, 6, 6, 6, 6,
       3, 6, 6, 6, 6, 6, 6, 3, 6, 6, 3, 6, 6, 6, 6, 6, 3, 3, 6, 6, 6, 6,
       6, 6, 3, 6, 3, 6, 6, 3, 3, 3, 6, 6, 3, 3, 6, 6, 3, 6, 6, 6, 6, 6,
       6, 6, 6, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 6, 6, 6, 6, 6, 6, 3, 3,
       6, 6, 6, 3, 3, 6, 3, 6, 6, 3, 3, 6, 3, 6, 6, 6, 6, 3, 6, 6, 3, 3,
       3, 6, 6, 6, 3, 3, 6, 3, 6, 6, 6, 6, 6, 6, 6, 3, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 6, 6, 6,

In [45]:
print("Accuracy on unknown data is",accuracy_score(y_test,y2))

Accuracy on unknown data is 0.1475237091675448
